In [20]:
import numpy as np
import random

In [21]:
def read_input_from_file(file_path):
  with open(file_path, 'r') as file:
    # Ler a primeira linha que contém N
    N = int(file.readline().strip())  # Quantidade de itens
    
    # Ler a segunda linha que contém C
    C = int(file.readline().strip())  # Capacidade de cada caixa
    
    # Ler os pesos dos itens, um por linha
    item_weights = []
    for _ in range(N):
      weight = int(file.readline().strip())
      item_weights.append(weight)
  
  return N, C, item_weights

In [22]:
def write_output_to_file(file_path, best_fitness):
  with open(file_path, 'w') as file:
    file.write(f'Número de bins usados (fitness): {best_fitness}\n')

In [23]:
# Função para gerar um voo de Lévy
def levy_flight(Lambda):
  sigma = (np.math.gamma(1 + Lambda) * np.sin(np.pi * Lambda / 2) / 
          (np.math.gamma((1 + Lambda) / 2) * Lambda * 2**((Lambda - 1) / 2))) ** (1 / Lambda)
  u = np.random.normal(0, sigma)
  v = np.random.normal(0, 1)
  step = u / abs(v) ** (1 / Lambda)
  return step

In [24]:
# Função para a heurística de First Fit
def first_fit(item_weights, bin_capacity):
  bins = []
  for weight in item_weights:
    placed = False
    for bin in bins:
      if sum(bin) + weight <= bin_capacity:
        bin.append(weight)
        placed = True
        break
    if not placed:
      bins.append([weight])
  return bins

In [25]:
# Função para calcular a aptidão (fitness)
def fitness(bins):
  return len(bins)

In [26]:
# Função principal ajustada para o formato de input (N, C) e lista de pesos A[i]
def adaptive_cuckoo_search_from_file(file_path, output_file, n_nests=20, pa=0.25, max_iter=20):
  N, C, item_weights = read_input_from_file(file_path)
  
  # lista = []

  for iter in range(max_iter):
    nests = [np.random.permutation(item_weights) for _ in range(n_nests)]
    best_nest = min(nests, key=lambda nest: fitness(first_fit(nest, C)))
    val = fitness(first_fit(best_nest, C))
    best_global = best_nest
    for i in range(n_nests):
      step_size = levy_flight(1.5)
      new_nest = nests[i] + step_size * (nests[i] - nests[np.random.randint(n_nests)])
      new_nest = np.clip(new_nest, 0, N-1)  # Garantir que permanece dentro dos limites

      # Aplicar ROV para transformar em permutação
      # random.shuffle(new_test)
      new_nest = np.argsort(new_nest)

      new_bins = first_fit(new_nest, C)
      if fitness(new_bins) < fitness(first_fit(nests[i], C)):
        nests[i] = new_nest

    # Abandonar parte dos ninhos com probabilidade pa
    for i in range(n_nests):
      if np.random.rand() < pa:
        nests[i] = np.random.permutation(item_weights)

    best_nest = min(nests, key=lambda nest: fitness(first_fit(nest, C)))
    nval = fitness(first_fit(best_nest, C))

    if nval < val:
      val = nval
      best_global = best_nest
    
    # lista.append(val)
    # print("iter:", iter, val)

  # Escrever o resultado no arquivo de saída
  write_output_to_file(output_file, fitness(first_fit(best_global, C)))

  # print(lista)

  return fitness(first_fit(best_global, C))

In [27]:
def main():
  input_file = 'input.in'
  output_file = 'output.out'
  best_fitness = adaptive_cuckoo_search_from_file(input_file, output_file)
  print(f'Resultado escrito no arquivo: {output_file}')

In [ ]:
if __name__ == "__main__":
  main()